
#utils

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
# from keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
from gensim.models import KeyedVectors

import gensim

import numpy as np

import itertools


def text_to_word_list(text):
    # Pre process and convert texts to a list of words
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

def make_w2v_embeddings(df, embedding_dim=300, empty_w2v=False):
    vocabs = {}
    vocabs_cnt = 0

    vocabs_not_w2v = {}
    vocabs_not_w2v_cnt = 0

    # Stopwords
    stops = set(stopwords.words('english'))

    # Load word2vec
    print("Loading word2vec model(it may takes 2-3 mins) ...")

    if empty_w2v:
        word2vec = EmptyWord2Vec
    else:
        # word2vec = KeyedVectors.load_word2vec_format("./data/GoogleNews-vectors-negative300.bin.gz", binary=True)
        word2vec = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz", binary=True)


    for index, row in df.iterrows():
        # Print the number of embedded sentences.
        if index != 0 and index % 1000 == 0:
            print("{:,} sentences embedded.".format(index), flush=True)

        # Iterate through the text of both questions of the row
        for question in ['question1', 'question2']:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
                # Check for unwanted words
                if word in stops:
                    continue

                # If a word is missing from word2vec model.
                if word not in word2vec.key_to_index:
                    if word not in vocabs_not_w2v:
                        vocabs_not_w2v_cnt += 1
                        vocabs_not_w2v[word] = 1

                # If you have never seen a word, append it to vocab dictionary.
                if word not in vocabs:
                    vocabs_cnt += 1
                    vocabs[word] = vocabs_cnt
                    q2n.append(vocabs_cnt)
                else:
                    q2n.append(vocabs[word])

            # Append question as number representation
            df.at[index, question + '_n'] = q2n

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabs.items():
        if word in word2vec.key_to_index:
            embeddings[index] = word2vec.get_vector(word)
    del word2vec

    return df, embeddings

def make_w2v_embeddings2(df, embedding_dim=300, empty_w2v=False):
    vocabs = {}
    vocabs_cnt = 0

    vocabs_not_w2v = {}
    vocabs_not_w2v_cnt = 0

    # Stopwords
    stops = set(stopwords.words('english'))

    # Load word2vec
    print("Loading word2vec model(it may takes 2-3 mins) ...")

    if empty_w2v:
        word2vec = EmptyWord2Vec
    else:
        word2vec = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/compairedataset/GoogleNews-vectors-negative300.bin.gz", binary=True)
        # word2vec = gensim.models.word2vec.Word2Vec.load("./data/Quora-Question-Pairs.w2v").wv

    for index, row in df.iterrows():
        # Print the number of embedded sentences.
        # if index != 0 and index % 1000 == 0:
        #     print("{:,} sentences embedded.".format(index), flush=True)

        # Iterate through the text of both questions of the row
        for question in ['question1', 'question2']:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
                # Check for unwanted words
                if word in stops:
                    continue

                # If a word is missing from word2vec model.
                if word not in word2vec.vocab:
                    if word not in vocabs_not_w2v:
                        vocabs_not_w2v_cnt += 1
                        vocabs_not_w2v[word] = 1

                # If you have never seen a word, append it to vocab dictionary.
                if word not in vocabs:
                    vocabs_cnt += 1
                    vocabs[word] = vocabs_cnt
                    q2n.append(vocabs_cnt)
                else:
                    q2n.append(vocabs[word])

            # Append question as number representation
            df.at[index, question + '_n'] = q2n

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabs.items():
        if word in word2vec.vocab:
            embeddings[index] = word2vec.word_vec(word)
    del word2vec

    return df, embeddings


def split_and_zero_padding(df, max_seq_length):
    # Split to dicts
    X = {'left': df['question1_n'], 'right': df['question2_n']}

    # Zero padding
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset


class EmptyWord2Vec:
    """
    Just for test use.
    """
    vocab = {}
    word_vec = {}
from tensorflow.keras.layers import Layer
import tensorflow as tf

class ManDist(Layer):
    def __init__(self, **kwargs):
        super(ManDist, self).__init__(**kwargs)

    def call(self, inputs):
        left, right = inputs
        return tf.exp(-tf.reduce_sum(tf.abs(left - right), axis=1, keepdims=True))


In [ ]:
from time import time
import pandas as pd

import matplotlib
from transformers import TFBertModel, BertTokenizer
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.python.keras.models import Model, Sequential
from keras.layers import Input, Embedding, LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout



TRAIN_CSV = '/content/drive/MyDrive/compairedataset/questions.csv'

# Load training set
train_df = pd.read_csv(TRAIN_CSV)
train_df = train_df.sample(n=10000, random_state=42)
train_df.reset_index(drop=True, inplace=True)
for q in ['question1', 'question2']:
    train_df[q + '_n'] = train_df[q]

# Make word2vec embeddings
embedding_dim = 300
max_seq_length = 20
use_w2v = True

# train_df, embeddings = make_w2v_embeddings(train_df, embedding_dim=embedding_dim, empty_w2v=not use_w2v)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')


# Split to train validation
validation_size = int(len(train_df) * 0.1)
training_size = len(train_df) - validation_size

X = train_df[['question1_n', 'question2_n']]
Y = train_df['is_duplicate']



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
from time import time
import pandas as pd

import matplotlib
from transformers import TFBertModel, BertTokenizer
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.python.keras.models import Model, Sequential
from keras.layers import Input, Embedding, LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout


# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
removes =[]
# for i,xx in enumerate(X["question1_n"]):
for i,xx in enumerate(X["question1_n"]):
  q1, q2 = X["question1_n"][i],X["question2_n"][i]
  if type(q1) != type("a") or type(q2) != type("a"):
    removes.append(i)

print(removes)

  # if (type(xx["question1_n"])!=type("a")) or (type(xx["question2_n"])!=type("a")):
  #   print(i)

[]


In [ ]:
X = X.drop(removes)
Y = Y.drop(removes)

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

In [ ]:
X1_pre, X2_pre = X_train["question1_n"], X_train["question2_n"]
#
X1_pre_val, X2_pre_val = X_validation["question1_n"], X_validation["question2_n"]


In [ ]:
tokinzed_data1_n = tokenizer(X1_pre.to_list(),  return_tensors="np", max_length= 20, padding="max_length", truncation=True)
tokinzed_data2_n = tokenizer(X2_pre.to_list(),  return_tensors="np", max_length= 20, padding="max_length", truncation=True)

In [ ]:
tokinzed_data1_n_val = tokenizer(X1_pre_val.to_list(),  return_tensors="np", max_length= 20, padding="max_length", truncation=True)
tokinzed_data2_n_val = tokenizer(X2_pre_val.to_list(),  return_tensors="np", max_length= 20, padding="max_length", truncation=True)

In [ ]:
inputs_ids_data1 = tokinzed_data1_n["input_ids"]
inputs_ids_data2 = tokinzed_data2_n["input_ids"]

masks_data1 = tokinzed_data1_n["attention_mask"]
masks_data2 = tokinzed_data1_n["attention_mask"]

# val

inputs_ids_data1_val = tokinzed_data1_n_val["input_ids"]
inputs_ids_data2_val = tokinzed_data2_n_val["input_ids"]

masks_data1_val = tokinzed_data1_n_val["attention_mask"]
masks_data2_val = tokinzed_data2_n_val["attention_mask"]


In [ ]:
print(inputs_ids_data1.shape)


(9000, 20)


In [ ]:
import numpy as np
np.save('/content/drive/MyDrive/compairedataset/inputs_ids_data1.npy',inputs_ids_data1)
np.save('/content/drive/MyDrive/compairedataset/inputs_ids_data2.npy',inputs_ids_data2)
np.save('/content/drive/MyDrive/compairedataset/masks_data1.npy',masks_data1)
np.save('/content/drive/MyDrive/compairedataset/masks_data2.npy',masks_data2)

np.save('/content/drive/MyDrive/compairedataset/inputs_ids_data1_val.npy',inputs_ids_data1_val)
np.save('/content/drive/MyDrive/compairedataset/inputs_ids_data2_val.npy',inputs_ids_data2_val)
np.save('/content/drive/MyDrive/compairedataset/masks_data1_val.npy',masks_data1_val)
np.save('/content/drive/MyDrive/compairedataset/masks_data2_val.npy',masks_data2_val)

In [ ]:
inputs_ids_data11=np.load('/content/drive/MyDrive/compairedataset/inputs_ids_data1.npy')
inputs_ids_data11.shape

(363913, 286)

In [ ]:
import numpy as np
inputs_ids_data1=np.load('/content/drive/MyDrive/compairedataset/inputs_ids_data1.npy')
inputs_ids_data2=np.load('/content/drive/MyDrive/compairedataset/inputs_ids_data2.npy')
masks_data1=np.load('/content/drive/MyDrive/compairedataset/masks_data1.npy')
masks_data2=np.load('/content/drive/MyDrive/compairedataset/masks_data2.npy')

inputs_ids_data1_val=np.load('/content/drive/MyDrive/compairedataset/inputs_ids_data1_val.npy')
inputs_ids_data2_val=np.load('/content/drive/MyDrive/compairedataset/inputs_ids_data2_val.npy')
masks_data1_val=np.load('/content/drive/MyDrive/compairedataset/masks_data1_val.npy')
masks_data2_val=np.load('/content/drive/MyDrive/compairedataset/masks_data2_val.npy')

In [ ]:
with tf.device('/gpu:0'):
    inputs_ids_data1_tensor = tf.convert_to_tensor(inputs_ids_data1)
    inputs_ids_data2_tensor = tf.convert_to_tensor(inputs_ids_data2)
    masks_data1_tensor = tf.convert_to_tensor(masks_data1)
    masks_data2_tensor = tf.convert_to_tensor(masks_data2)

    inputs_ids_data1_val_tensor = tf.convert_to_tensor(inputs_ids_data1_val)
    inputs_ids_data2_val_tensor = tf.convert_to_tensor(inputs_ids_data2_val)
    masks_data1_val_tensor = tf.convert_to_tensor(masks_data1_val)
    masks_data2_val_tensor = tf.convert_to_tensor(masks_data2_val)

In [ ]:
del inputs_ids_data1
del inputs_ids_data2
del masks_data1
del masks_data2

del inputs_ids_data1_val
del inputs_ids_data2_val
del masks_data1_val
del masks_data2_val

In [ ]:
import gc
gc.collect()

476

In [ ]:
print(inputs_ids_data1_val.shape)
print(inputs_ids_data1.shape)

(40435, 286)
(363913, 286)


In [ ]:
inputs_ids_data2.shape

#with bert

In [ ]:
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer, LSTM
import tensorflow as tf




class BertLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = bert_model

    def call(self, inputs, masks):
        return self.bert(inputs, masks)[0]


class SelfAttention(Layer):
    def __init__(self, units, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.W_q = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='random_normal',
                                   trainable=True)
        self.W_k = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='random_normal',
                                   trainable=True)
        self.W_v = self.add_weight(shape=(input_shape[-1], self.units),
                                   initializer='random_normal',
                                   trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = tf.tensordot(inputs, self.W_q, axes=1)
        key = tf.tensordot(inputs, self.W_k, axes=1)
        value = tf.tensordot(inputs, self.W_v, axes=1)
        scores = tf.matmul(query, key, transpose_b=True)
        scores /= tf.math.sqrt(tf.cast(tf.shape(key)[-1], tf.float32))
        attention_weights = tf.nn.softmax(scores, axis=-1)
        context_vector = tf.matmul(attention_weights, value)
        return context_vector


max_seq_length = 20
batch_size = 256
n_epoch = 10

left_input = Input(shape=(max_seq_length,), dtype='int32', name='left_input')
right_input = Input(shape=(max_seq_length,), dtype='int32', name='right_input')

left_input_mask = Input(shape=(max_seq_length,), dtype='int32', name='left_input_masks')
right_input_mask = Input(shape=(max_seq_length,), dtype='int32', name='right_input_masks')

bert_layer = BertLayer(bert_model)#BertLayer(bert_model)
bert_output_left = bert_layer(left_input, left_input_mask)
bert_output_right =bert_layer(right_input, right_input_mask)


attention_units = 64
lstm_units = 50

attention = SelfAttention(attention_units)

attention_left = attention(bert_output_left)
attention_right = attention(bert_output_right)

LSTM_cell = LSTM(lstm_units, return_sequences= True)

lstm_left = LSTM_cell(attention_left)
lstm_right = LSTM_cell(attention_right)

pooled_left = tf.keras.layers.GlobalAveragePooling1D()(lstm_left)
pooled_right = tf.keras.layers.GlobalAveragePooling1D()(lstm_right)

malstm_distance = ManDist()([pooled_left, pooled_right])


model = Model(inputs=[left_input, right_input, left_input_mask, right_input_mask], outputs=[malstm_distance])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

model.summary()

training_start_time = time()
atten_malstm_trained = model.fit([inputs_ids_data1, inputs_ids_data2, masks_data1, masks_data2 ], Y_train,
                                 batch_size=batch_size, epochs=n_epoch,
                                 validation_data=([inputs_ids_data1_val, inputs_ids_data2_val, masks_data1_val, masks_data2_val], Y_validation))
# atten_malstm_trained = model.fit(
#     [inputs_ids_data1_tensor, inputs_ids_data2_tensor, masks_data1_tensor, masks_data2_tensor],  # Use GPU tensors
#     Y_train,
#     batch_size=batch_size,
#     epochs=n_epoch,
#     validation_data=(
#         [inputs_ids_data1_val_tensor, inputs_ids_data2_val_tensor, masks_data1_val_tensor, masks_data2_val_tensor],  # Use GPU tensors
#         Y_validation
#     )
# )
training_end_time = time()

print("Training time finished.\n%d epochs in %12.2f" % (n_epoch, training_end_time - training_start_time))
model.save('./data/bert_atten_SiameseLSTM.h5')


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 left_input (InputLayer)     [(None, 20)]                 0         []                            
                                                                                                  
 left_input_masks (InputLay  [(None, 20)]                 0         []                            
 er)                                                                                              
                                                                                                  
 right_input (InputLayer)    [(None, 20)]                 0         []                            
                                                                                                  
 right_input_masks (InputLa  [(None, 20)]                 0         []                      

5/5 [==============================] - 328s 53s/step - loss: 0.3569 - accuracy: 0.5798 - val_loss: 0.4834 - val_accuracy: 0.3780
Epoch 2/10
5/5 [==============================] - 250s 49s/step - loss: 0.3678 - accuracy: 0.5401 - val_loss: 0.3435 - val_accuracy: 0.5830
Epoch 3/10
5/5 [==============================] - 248s 49s/step - loss: 0.4925 - accuracy: 0.4563 - val_loss: 0.6052 - val_accuracy: 0.3770
Epoch 4/10
5/5 [==============================] - 250s 49s/step - loss: 0.4509 - accuracy: 0.4604 - val_loss: 0.3596 - val_accuracy: 0.5950
Epoch 5/10
5/5 [==============================] - 250s 49s/step - loss: 0.5364 - accuracy: 0.4206 - val_loss: 0.3490 - val_accuracy: 0.4980
Epoch 6/10
5/5 [==============================] - 249s 49s/step - loss: 0.4159 - accuracy: 0.5247 - val_loss: 0.5908 - val_accuracy: 0.3770
Epoch 7/10
